# **Final Project COMPSCI 4411/9538**


## Data Preparation

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
import shap
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import os
import urllib.parse
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
import xgboost as xgb
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, cohen_kappa_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from kmodes.kmodes import KModes
import os
import urllib.parse
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
import xgboost as xgb
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


In [ ]:

# URL of the file to download
url = "https://s.mx.sb/s/RZr8i3ewjkJr5RT/download/child-mind-institute-problematic-internet-use.zip"

# Parse the filename from the URL path
parsed_url = urllib.parse.urlparse(url)
filename = os.path.basename(parsed_url.path)

# Check if the file already exists
if not os.path.exists(filename):
    print(f"File {filename} does not exist. Downloading...")

    # Download the file using curl with progress bar
    !curl -L --progress-bar -o "{filename}" "{url}"

# Unzip the file if it exists
if os.path.exists(filename):
    print(f"Unzipping {filename}...")
    !unzip -o "{filename}"
else:
    print(f"File {filename} was not downloaded or is missing.")


## EDA


#### 1) Dataset preview

In [ ]:
train = pd.read_csv('train.csv')
train = train.drop(columns="id")
dict = pd.read_csv('data_dictionary.csv')
print(train.shape)
print(train.columns)

In [ ]:
dict.head()

In [ ]:
dict[
dict['Type'].str.contains('int') & ~dict['Type'].str.contains('categorical')
]

In [ ]:
print(dict[
dict['Field'].str.contains('PreInt_EduHx-computerinternet_hoursday')
]["Value Labels"].iloc[0])

#### 2) Missing data

In [ ]:
# Create a bar plot to visualize the total null values in each feature

missing_count = train.isnull().sum().to_frame('null_count')
missing_count['feature'] = missing_count.index
missing_count['null_ratio'] = missing_count['null_count'] / len(train)
missing_count = missing_count.sort_values('null_count', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x='feature', y='null_count', data=missing_count, palette='rainbow', legend=False)
plt.xticks(rotation=90)
plt.title('Total Null Values in train_df')
plt.tight_layout()
plt.show()


In [ ]:
# Null values ​​in data with no missing values ​​in Sii
target = train[train['sii'].notnull()]

missing_count = target.isnull().sum().to_frame('null_count')
missing_count['feature'] = missing_count.index
missing_count['null_ratio'] = missing_count['null_count'] / len(target)
missing_count = missing_count.sort_values('null_count', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x='feature', y='null_count', data=missing_count, palette='rainbow', legend=False)
plt.xticks(rotation=90)
plt.title('Train_df without null values ​​in sii')
plt.tight_layout()
plt.show()

#### 3) Correlation Analysis

In [ ]:
# Define a list of columns to be extracted from the dataset for analysis.
columns_to_include = [
    'PCIAT-PCIAT_Total', 'Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score', 'Physical-BMI',
    'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
    'Physical-Diastolic_BP', 'Physical-Systolic_BP', 'Physical-HeartRate',
    'PreInt_EduHx-computerinternet_hoursday', 'SDS-SDS_Total_T', 'PAQ_A-PAQ_A_Total',
    'PAQ_C-PAQ_C_Total', 'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins',
    'Fitness_Endurance-Time_Sec', 'FGC-FGC_CU', 'FGC-FGC_GSND', 'FGC-FGC_GSD',
    'FGC-FGC_PU', 'FGC-FGC_SRL', 'FGC-FGC_SRR', 'FGC-FGC_TL', 'BIA-BIA_Activity_Level_num',
    'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW',
    'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
    'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW'
]


# Filter the dataset to include only rows where 'sii' is not null and select the specified columns.
filtered_data = train[train['sii'].notnull()][columns_to_include]
# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(filtered_data.corr(), annot=True, fmt=".2f", cmap='rainbow')
plt.show()


#### 4) Features used in this analysis - Univariate Analysis & and Multivariate Analysis

In [ ]:
# Select relevant columns (FitnessGram Child, PCIAT)
pciat_columns = [col for col in train.columns if 'PCIAT-' in col]

# Combine all relevant columns
df = train[pciat_columns]

#drop season columns
season_columns = [col for col in df.columns if 'Season' in col]
df = df.drop(columns=season_columns)


# Numeric Data Visualization
df.hist(bins=15, figsize=(15, 10), layout=(3, 7))
plt.show()

# Categorical Data Visualization
for column in df.select_dtypes(include='object').columns:
    plt.figure(figsize=(10, 4))
    sns.countplot(y=column, data=df)
    plt.title(f'Distribution of {column}')
    plt.show()


In [ ]:
df.select_dtypes(include=['float64', 'int64']).plot(kind='box', subplots=True, layout=(3, 7), figsize=(12, 10), sharex=False, sharey=False)
plt.show()

In [ ]:
# Select relevant columns (BIA)
bia_columns = [col for col in train.columns if 'BIA-' in col]

# Combine all relevant columns
df = train[bia_columns]

#drop season columns
season_columns = [col for col in df.columns if 'Season' in col]
df = df.drop(columns=season_columns)


# Numeric Data Visualization
df.hist(bins=15, figsize=(15, 10), layout=(3, 7))
plt.show()

# Categorical Data Visualization
for column in df.select_dtypes(include='object').columns:
    plt.figure(figsize=(10, 4))
    sns.countplot(y=column, data=df)
    plt.title(f'Distribution of {column}')
    plt.show()


In [ ]:
df.select_dtypes(include=['float64', 'int64']).plot(kind='box', subplots=True, layout=(3, 7), figsize=(12, 10), sharex=False, sharey=False)
plt.show()

In [ ]:
# Select relevant columns (FGC)
fgc_columns = [col for col in train.columns if 'FGC' in col]

# Combine all relevant columns
df = train[fgc_columns]

#drop season columns
season_columns = [col for col in df.columns if 'Season' in col]
df = df.drop(columns=season_columns)


# Numeric Data Visualization
df.hist(bins=15, figsize=(15, 10), layout=(3, 7))
plt.show()

# Categorical Data Visualization
for column in df.select_dtypes(include='object').columns:
    plt.figure(figsize=(10, 4))
    sns.countplot(y=column, data=df)
    plt.title(f'Distribution of {column}')
    plt.show()

In [ ]:
# Box plots for all numeric variables
df.select_dtypes(include=['float64', 'int64']).plot(kind='box', subplots=True, layout=(3, 7), figsize=(12, 10), sharex=False, sharey=False)
plt.show()


#### 5） Target Variable Analysis

In [ ]:
# Distribution of SII Target Variable

# Define the mapping for `sii` levels including "Missing" for NaNs
sii_map = {
    0: "None",
    1: "Mild",
    2: "Moderate",
    3: "Severe",
    "missing": "Missing Data"
}

# Replace NaN values with "missing" to count them as a separate category
train['sii_filled'] = train['sii'].fillna("missing")
train['sii_label'] = train['sii_filled'].map(sii_map)

# Count the occurrences for each `sii` level, including missing data
sii_counts = train['sii_label'].value_counts()
sii_percentages = (sii_counts / sii_counts.sum()) * 100

# Create DataFrame for plotting with counts and percentages, placing "Missing Data" at the end
sii_data = pd.DataFrame({
    'SII Level': sii_counts.index,
    'Count': sii_counts.values,
    'Percentage': sii_percentages.values
})

# Reorder to ensure "Missing Data" is at the end
sii_data['SII Level'] = pd.Categorical(
    sii_data['SII Level'],
    categories=["None", "Mild", "Moderate", "Severe", "Missing Data"],
    ordered=True
)
sii_data = sii_data.sort_values('SII Level')

# Define color mapping based on sii levels
colors = {
    "None": "#4B9CD3",  # Blue
    "Mild": "#0074D9",  # Darker Blue
    "Moderate": "#17BECF",  # Teal
    "Severe": "#FF4136",  # Red
    "Missing Data": "#A9A9A9"  # Gray
}

# Plot the data using matplotlib
bars = plt.bar(sii_data['SII Level'], sii_data['Count'], color=[colors[level] for level in sii_data['SII Level']])

# Add count labels to each bar
for bar, count in zip(bars, sii_data['Count']):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 5, str(count),
             ha='center', va='bottom', color='white', fontsize=10, fontweight='bold')

# Set the title and labels to match previous settings
plt.title("Distribution of SII Target Variable", fontsize=20, color="#004080")
plt.xlabel("SII Severity Level", fontsize=14)
plt.ylabel("Number of Participants", fontsize=14)

# Customize the plot appearance
plt.gca().set_facecolor('white')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(False)
plt.tight_layout()

# Show the plot
plt.show()


## Unsupervised Learning

#### Data Preprocessing

In [ ]:
df = train

# Define the threshold for missing value percentage
threshold = 0.5

# Calculate the percentage of missing values in each column
missing_percentage = df.isnull().sum() / len(df)

# Drop columns with >50% missing values
columns_to_remove = missing_percentage[missing_percentage > threshold].index.tolist()

# Remove the identified columns from the DataFrame
df_cleaned = df.drop(columns=columns_to_remove)

# Check the structure of the cleaned DataFrame
print("Structure of the cleaned DataFrame:\n", df_cleaned.info())

# Check for missing values in the cleaned DataFrame
missing_values_cleaned = df_cleaned.isnull().sum()
print("\nMissing values in the cleaned DataFrame:\n", missing_values_cleaned)

### Cluster 1 - PCIAT

In [ ]:
# Select relevant columns (FitnessGram Child, PCIAT)
pciat_columns = [col for col in df_cleaned.columns if 'PCIAT-' in col]

# Combine all relevant columns
filtered_data = df_cleaned[pciat_columns]

#drop season columns
season_columns = [col for col in filtered_data.columns if 'Season' in col]
filtered_data = filtered_data.drop(columns=season_columns)
filtered_data = filtered_data.drop(columns='PCIAT-PCIAT_Total')
# Inspect missing values in the selected dataset
print("Missing Values in Selected Data:")
print(filtered_data.isnull().sum().sort_values(ascending=False))

##### 1) KNN & K-means

In [ ]:
# Define the range of neighbors to test
neighbor_values = [2,3,5,7,10]

# Store results for evaluation
imputed_datasets = {}

for n in neighbor_values:
    # Initialize the KNN Imputer with different numbers of neighbors
    knn_imputer = KNNImputer(n_neighbors=n)

    # Perform imputation
    imputed_data = knn_imputer.fit_transform(filtered_data)

    # Use the actual columns after imputation which are numbered
    imputed_datasets[n] = pd.DataFrame(imputed_data)
    print(f"Imputation complete for n_neighbors={n}")

In [ ]:
# Define the range of max_iter values to test
max_iter_values = [5, 10, 20]

# Store results for evaluation
imputed_datasets_iterative = {}

for max_iter in max_iter_values:
    # Initialize the Iterative Imputer with different max_iter values
    iterative_imputer = IterativeImputer(max_iter=max_iter, random_state=0)

    # Perform imputation
    imputed_data = iterative_imputer.fit_transform(filtered_data)

    # Store the imputed dataset
    imputed_datasets_iterative[max_iter] = pd.DataFrame(imputed_data, columns = filtered_data.columns)
    print(f"Iterative Imputation complete for max_iter={max_iter}")


# Replace with the index of your column of interest
column_index = 0
column_name = filtered_data.columns[column_index]

plt.figure(figsize=(12, 8))

# Plot distributions for KNN Imputation
for n, imputed_df in imputed_datasets.items():
    sns.kdeplot(imputed_df[column_index], label=f'KNN (n_neighbors={n})')

# Plot distributions for Iterative Imputation
for max_iter, imputed_df in imputed_datasets_iterative.items():
    sns.kdeplot(imputed_df[column_name], label=f'Iterative (max_iter={max_iter})')

# Plot the original distribution (if you have it)
sns.kdeplot(filtered_data[column_name], label='Original Data')


plt.xlabel(column_name)
plt.ylabel('Density')
plt.title(f'Distribution of {column_name} for Different Imputation Methods')
plt.legend()
plt.show()

In [ ]:
# As they are all the same, we then choose to use KNN when n=5 to fill the missing values in relevant columns for PCIAT
# and drop the column includes seasons

# Initialize the KNN Imputer with n_neighbors=5
knn_imputer = KNNImputer(n_neighbors=5)

# Perform imputation
filtered_data_imputed = knn_imputer.fit_transform(filtered_data)

# Convert the imputed data back to a DataFrame
filtered_data_imputed = pd.DataFrame(filtered_data_imputed, columns=filtered_data.columns)

print(filtered_data_imputed.head())

In [ ]:
# Perform imputation
filtered_data_imputed = knn_imputer.fit_transform(filtered_data)

# Convert the imputed data back to a DataFrame
filtered_data_imputed = pd.DataFrame(filtered_data_imputed, columns=filtered_data.columns)

print(filtered_data_imputed.head())

In [ ]:
# Do clustering for the relevant columns for FGC in filtered_data_imputed and visualized clustering and elbow
# Scale the data
scaled_data = filtered_data_imputed

# Calculate the silhouette score for different numbers of clusters
range_n_clusters = range(2, 11)
silhouette_scores = []

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(scaled_data)
    silhouette_avg = silhouette_score(scaled_data, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"For n_clusters = {n_clusters}, the average silhouette_score is : {silhouette_avg}")

# Plot the silhouette scores
plt.plot(range_n_clusters, silhouette_scores, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis for Optimal k')
plt.show()


In [ ]:
# Elbow method
inertia = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=0)
    kmeans.fit(scaled_data)
    inertia.append(kmeans.inertia_)

plt.plot(range(1, 11), inertia, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
# Based on the silhouette scores and elbow method, choose the optimal number of clusters
optimal_k = 3

# Perform KMeans clustering with the optimal k
kmeans = KMeans(n_clusters=optimal_k, random_state=0)
filtered_data_imputed['cluster'] = kmeans.fit_predict(scaled_data)
filtered_data_imputed.head()

##### 2) DBSCAN

In [ ]:
# Try different epsilon and min_samples values
eps_values = [0.5, 1, 1.5, 2]
min_samples_values = [5, 10, 15]

best_eps = None
best_min_samples = None
best_silhouette = -1

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(scaled_data)

        # Check for only one cluster
        if len(set(labels)) > 1:
          silhouette = silhouette_score(scaled_data, labels)

          print(f"eps={eps}, min_samples={min_samples}, Silhouette Score: {silhouette}")

          if silhouette > best_silhouette:
              best_silhouette = silhouette
              best_eps = eps
              best_min_samples = min_samples

# Apply DBSCAN with the best parameters found
dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
filtered_data_imputed['cluster'] = dbscan.fit_predict(scaled_data)

# Analyze the clusters
print(filtered_data_imputed.groupby('cluster').mean())

# Calculate Silhouette Score for the best model
print(f"Best Silhouette Score (DBSCAN): {best_silhouette}")

In [ ]:
# Visualize the clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(x=filtered_data_imputed.columns[0], y=filtered_data_imputed.columns[1], hue='cluster', data=filtered_data_imputed, palette='viridis')
plt.title('DBSCAN Clustering Results')
plt.show()

##### 3) GMM

In [ ]:
# Test different numbers of components
n_components_range = range(2, 6)

best_n_components = None
best_silhouette = -1

for n_components in n_components_range:
    gmm = GaussianMixture(n_components=n_components, random_state=0)
    labels = gmm.fit_predict(scaled_data)
    silhouette = silhouette_score(scaled_data, labels)

    print(f"n_components={n_components}, Silhouette Score: {silhouette}")

    if silhouette > best_silhouette:
        best_silhouette = silhouette
        best_n_components = n_components

# Apply GMM with the best number of components
gmm = GaussianMixture(n_components=best_n_components, random_state=0)
filtered_data_imputed['cluster'] = gmm.fit_predict(scaled_data)

# Analyze clusters
print(filtered_data_imputed.groupby('cluster').mean())

# Calculate Silhouette Score for the best model
print(f"Best Silhouette Score (GMM): {best_silhouette}")

In [ ]:
# Visualize the clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(x=filtered_data_imputed.columns[0], y=filtered_data_imputed.columns[1], hue='cluster', data=filtered_data_imputed, palette='viridis')
plt.title('GMM Clustering Results')
plt.show()

### Cluster 2 - BIA


In [ ]:
# Select relevant columns starting with 'FGC' and ending with 'zone'
bia_columns = [col for col in df_cleaned.columns if col.startswith('BIA')]
filtered_data_bia = df_cleaned[bia_columns]

#drop season columns
season_columns = [col for col in filtered_data_bia.columns if 'Season' in col]
filtered_data_bia = filtered_data_bia.drop(columns=season_columns)

# Inspect missing values in the selected dataset
print("Missing Values in Selected Data:")
print(filtered_data_bia.isnull().sum().sort_values(ascending=False))

In [ ]:
# Perform one-hot encoding on 'BIA-BIA_Frame_num'

# One-hot encode the column
bia_encoded = pd.get_dummies(filtered_data_bia['BIA-BIA_Frame_num'], prefix='BIA_Frame_num')

# Concatenate the one-hot encoded columns with the original DataFrame
filtered_data_bia = pd.concat([filtered_data_bia, bia_encoded], axis=1)

# Drop the original 'BIA-BIA_Frame_num' column
filtered_data_bia = filtered_data_bia.drop('BIA-BIA_Frame_num', axis=1)

print(filtered_data_bia.head())

In [ ]:
# Identify continuous variables (numeric columns)
continuous_cols = filtered_data_bia.select_dtypes(include=['number']).columns

# Normalize using StandardScaler
scaler = StandardScaler()
filtered_data_bia[continuous_cols] = scaler.fit_transform(filtered_data_bia[continuous_cols])

print(filtered_data_bia.head())

#### 1) KNN & Kmodes

In [ ]:
# Perform imputation
filtered_data_imputed_bia = knn_imputer.fit_transform(filtered_data_bia)

# Convert the imputed data back to a DataFrame
filtered_data_imputed_bia = pd.DataFrame(filtered_data_imputed_bia, columns=filtered_data_bia.columns)

print(filtered_data_imputed_bia.head())

In [ ]:
scaled_data_bia = filtered_data_imputed_bia

# Calculate the silhouette score for different numbers of clusters
range_n_clusters = range(2, 11)
silhouette_scores = []

for n_clusters in range_n_clusters:
    kmodes = KModes(n_clusters=n_clusters, random_state=0)
    cluster_labels = kmodes.fit_predict(scaled_data_bia)
    silhouette_avg = silhouette_score(scaled_data_bia, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"For n_clusters = {n_clusters}, the average silhouette_score is : {silhouette_avg}")

# Plot the silhouette scores
plt.plot(range_n_clusters, silhouette_scores, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis for Optimal k')
plt.show()

In [ ]:
# Based on the silhouette scores and elbow method, choose the optimal number of clusters
optimal_k = 4

# Perform KMeans clustering with the optimal k
kmodes = KModes(n_clusters=optimal_k, random_state=0)
filtered_data_imputed_bia['cluster_bia'] = kmodes.fit_predict(scaled_data_bia)
filtered_data_imputed_bia.head()

#### 2) DBSCAN

In [ ]:
eps_values = [0.5, 1, 1.5, 2]
min_samples_values = [5, 10, 15]

best_eps = None
best_min_samples = None
best_silhouette = -1

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(scaled_data_bia)

        # Check for only one cluster
        if len(set(labels)) > 1:
          silhouette = silhouette_score(scaled_data_bia, labels)

          print(f"eps={eps}, min_samples={min_samples}, Silhouette Score: {silhouette}")

          if silhouette > best_silhouette:
              best_silhouette = silhouette
              best_eps = eps
              best_min_samples = min_samples

# Apply DBSCAN with the best parameters found
dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
filtered_data_imputed_bia['cluster_bia'] = dbscan.fit_predict(scaled_data_bia)

# Analyze the clusters
print(filtered_data_imputed_bia.groupby('cluster_bia').mean())
# Calculate Silhouette Score for the best model
print(f"Best Silhouette Score (DBSCAN): {best_silhouette}")

#### 3) GMM

In [ ]:
# Test different numbers of components
n_components_range = range(2, 6)

best_n_components = None
best_silhouette = -1

for n_components in n_components_range:
    gmm = GaussianMixture(n_components=n_components, random_state=0)
    labels = gmm.fit_predict(scaled_data_bia)
    silhouette = silhouette_score(scaled_data_bia, labels)

    print(f"n_components={n_components}, Silhouette Score: {silhouette}")

    if silhouette > best_silhouette:
        best_silhouette = silhouette
        best_n_components = n_components

# Apply GMM with the best number of components
gmm = GaussianMixture(n_components=best_n_components, random_state=0)
filtered_data_imputed_bia['cluster_bia'] = gmm.fit_predict(scaled_data_bia)

# Analyze clusters
print(filtered_data_imputed_bia.groupby('cluster_bia').mean())

# Calculate Silhouette Score for the best model
print(f"Best Silhouette Score (GMM): {best_silhouette}")

### Cluster 3 - FGC

In [ ]:
fgc_columns = [col for col in df_cleaned.columns if col.startswith('FGC') and col.endswith('Zone')]
filtered_data_fgc = df_cleaned[fgc_columns]

#drop season columns
season_columns = [col for col in filtered_data_fgc.columns if 'Season' in col]
filtered_data_fgc = filtered_data_fgc.drop(columns=season_columns)

# Inspect missing values in the selected dataset
print("Missing Values in Selected Data:")
print(filtered_data_fgc.isnull().sum().sort_values(ascending=False))

In [ ]:
filtered_data_imputed_fgc = knn_imputer.fit_transform(filtered_data_fgc)

# Convert the imputed data back to a DataFrame
filtered_data_imputed_fgc = pd.DataFrame(filtered_data_imputed_fgc, columns=filtered_data_fgc.columns)

print(filtered_data_imputed_fgc.head())

#### 1) KNN & KModes

In [ ]:
scaled_data_fgc = filtered_data_imputed_fgc

# Calculate the silhouette score for different numbers of clusters
range_n_clusters = range(2, 11)
silhouette_scores = []

for n_clusters in range_n_clusters:
    kmodes = KModes(n_clusters=n_clusters, random_state=0)
    cluster_labels = kmodes.fit_predict(scaled_data_fgc)
    silhouette_avg = silhouette_score(scaled_data_fgc, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"For n_clusters = {n_clusters}, the average silhouette_score is : {silhouette_avg}")

# Plot the silhouette scores
plt.plot(range_n_clusters, silhouette_scores, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis for Optimal k')
plt.show()

In [ ]:
# Elbow method for KModes
cost = []
K = range(1,11)
for num_clusters in list(K):
    kmode = KModes(n_clusters=num_clusters, init = "Cao", n_init = 1, verbose=1)
    kmode.fit_predict(scaled_data_fgc)
    cost.append(kmode.cost_)

plt.plot(K, cost, 'bx-')
plt.xlabel('No. of clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
optimal_k = 5

# Perform KMeans clustering with the optimal k
kmodes = KModes(n_clusters=optimal_k, random_state=0)
filtered_data_imputed_fgc['cluster_fgc'] = kmodes.fit_predict(scaled_data_fgc)
filtered_data_imputed_fgc.head()

#### 2) DBSCAN

In [ ]:
eps_values = [0.5, 1, 1.5, 2]
min_samples_values = [5, 10, 15]

best_eps = None
best_min_samples = None
best_silhouette = -1

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(scaled_data_fgc)

        # Check for only one cluster
        if len(set(labels)) > 1:
          silhouette = silhouette_score(scaled_data_fgc, labels)

          print(f"eps={eps}, min_samples={min_samples}, Silhouette Score: {silhouette}")

          if silhouette > best_silhouette:
              best_silhouette = silhouette
              best_eps = eps
              best_min_samples = min_samples

# Apply DBSCAN with the best parameters found
dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
filtered_data_imputed_fgc['cluster_fgc'] = dbscan.fit_predict(scaled_data_fgc)

# Analyze the clusters
print(filtered_data_imputed_fgc.groupby('cluster_fgc').mean())
# Calculate Silhouette Score for the best model
print(f"Best Silhouette Score (DBSCAN): {best_silhouette}")

#### 3）GMM

In [ ]:
# Test different numbers of components
n_components_range = range(2, 6)

best_n_components = None
best_silhouette = -1

for n_components in n_components_range:
    gmm = GaussianMixture(n_components=n_components, random_state=0)
    labels = gmm.fit_predict(scaled_data_fgc)
    silhouette = silhouette_score(scaled_data_fgc, labels)

    print(f"n_components={n_components}, Silhouette Score: {silhouette}")

    if silhouette > best_silhouette:
        best_silhouette = silhouette
        best_n_components = n_components

# Apply GMM with the best number of components
gmm = GaussianMixture(n_components=best_n_components, random_state=0)
filtered_data_imputed_fgc['cluster_fgc'] = gmm.fit_predict(scaled_data_fgc)

# Analyze clusters
print(filtered_data_imputed_fgc.groupby('cluster_fgc').mean())
print(filtered_data_imputed_fgc)
# Calculate Silhouette Score for the best model
print(f"Best Silhouette Score (GMM): {best_silhouette}")

## Supervised Learning

#### 1) Data Preparation

In [ ]:
# Fill missing values in columns starting with 'Physical' with the median
physical_columns = [col for col in df_cleaned.columns if col.startswith('Physical')]
filtered_data_su = df_cleaned[physical_columns]

# Drop the column that has Season so that all missing value will be filled with median and do train-spilt
season_columns = [col for col in filtered_data_su.columns if 'Season' in col]
filtered_data_su = filtered_data_su.drop(columns=season_columns)
print("Missing Values in Selected Data:")
print(filtered_data_su.isnull().sum().sort_values(ascending=False))

In [ ]:
# Fill missing values with the median for each column
for col in filtered_data_su.columns:
    median_value = filtered_data_su[col].median()
    filtered_data_su[col].fillna(median_value, inplace=True)

print("Missing Values after Median Imputation:")
print(filtered_data_su.isnull().sum().sort_values(ascending=False))

# Use median to filled the missing value for sii
df_cleaned['sii'].fillna(df_cleaned['sii'].median(), inplace=True)


In [ ]:
# Use the cluster labels as new features
filtered_data_su['cluster'] = kmeans.fit_predict(scaled_data)
filtered_data_su['cluster_bia'] = dbscan.fit_predict(scaled_data_bia)
filtered_data_su['cluster_fgc'] = gmm.fit_predict(scaled_data_fgc)
print(filtered_data_su.head())

In [48]:
# Define features (X) and target (y)
X = filtered_data_su
y = df_cleaned['sii']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check the class distribution after SMOTE
print(np.bincount(y_train))

#### 2) Baseline - Logistic Regression

In [ ]:
# Initialize Logistic Regression model with multi_class set to 'multinomial'
baseline_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial')
baseline_pipeline = make_pipeline(StandardScaler(), baseline_model)
baseline_pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred_base = baseline_pipeline.predict(X_test)
y_score_base = baseline_pipeline.predict_proba(X_test)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_base)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Binarize y_test if multiclass
y_test_bin = label_binarize(y_test, classes=[0, 1, 2, 3])
fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_score_base.ravel())
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()

In [ ]:
n_classes = 4

# Store FPR, TPR, and AUC for each class
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score_base[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'purple']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

# Plot the diagonal (random classifier)
plt.plot([0, 1], [0, 1], 'k--', lw=2)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curves')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# Calculate metrics
accuracy_base = accuracy_score(y_test, y_pred_base)
precision_base = precision_score(y_test, y_pred_base, average='weighted', zero_division=0)
recall_base = recall_score(y_test, y_pred_base, average='weighted')
f1_base = f1_score(y_test, y_pred_base, average='weighted')
kappa_base = cohen_kappa_score(y_test, y_pred_base)
roc_auc_base = roc_auc_score(label_binarize(y_test, classes=[0, 1, 2, 3]), y_score_base, average='macro', multi_class='ovr')

# Output the metrics
print(f"Accuracy: {accuracy_base:}")
print(f"Precision: {precision_base:}")
print(f"Recall: {recall_base:}")
print(f"F1 Score: {f1_base:}")
print(f"Cohen's Kappa: {kappa_base:}")
print(f"ROC-AUC: {roc_auc_base:}")

#### 3) Random Forest

In [ ]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200, 500],  
    'max_depth': [None, 10, 20, 30],     
    'min_samples_split': [2, 5, 10],    
}

# Initialize Random Forest
rf = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Output the best parameters and accuracy
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Accuracy: {grid_search.best_score_}")

In [56]:
# Create a RandomForestClassifier
rf_classifier = RandomForestClassifier(max_depth=20, min_samples_split=2, n_estimators=500, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_classifier.predict(X_test)
y_score_rf = rf_classifier.predict_proba(X_test)


In [ ]:
# Feature Importance Plot
feature_importances = rf_classifier.feature_importances_
feature_names = X.columns

# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Feature Importance from Random Forest')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# SHAP values for model interpretability
explainer = shap.TreeExplainer(rf_classifier)
shap_values = explainer.shap_values(X_test.iloc[:100,:])
shap.summary_plot(shap_values, X_test.iloc[:100,:], plot_type="bar")

In [ ]:
# Binarize y_test if multiclass
y_test_bin = label_binarize(y_test, classes=[0, 1, 2, 3])
fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_score_rf.ravel())
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
n_classes = 4

# Store FPR, TPR, and AUC for each class
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score_rf[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'purple']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

# Plot the diagonal (random classifier)
plt.plot([0, 1], [0, 1], 'k--', lw=2)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curves')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Calculate metrics
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted', zero_division=0)
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
kappa_rf = cohen_kappa_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(label_binarize(y_test, classes=[0, 1, 2, 3]), y_score_rf, average='macro', multi_class='ovr')

# Output the metrics
print(f"Accuracy: {accuracy_rf:}")
print(f"Precision: {precision_rf:}")
print(f"Recall: {recall_rf:}")
print(f"F1 Score: {f1_rf:}")
print(f"Cohen's Kappa: {kappa_rf:}")
print(f"ROC-AUC: {roc_auc_rf:}")

#### 3) XGBoost

In [63]:
# Initialize and train the XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred_xgb = xgb_model.predict(X_test)
y_score_xgb = xgb_model.predict_proba(X_test)


In [ ]:
# Feature Importance Plot
feature_importances = xgb_model.feature_importances_
feature_names = X.columns

# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Feature Importance from XGboost')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_xgb)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# SHAP values for model interpretability
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test.iloc[:100,:])
shap.summary_plot(shap_values, X_test.iloc[:100,:], plot_type="bar")

In [ ]:
# Binarize y_test if multiclass
y_test_bin = label_binarize(y_test, classes=[0, 1, 2, 3])
fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_score_xgb.ravel())
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


In [ ]:
n_classes = 4

# Store FPR, TPR, and AUC for each class
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score_xgb[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'purple']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

# Plot the diagonal (random classifier)
plt.plot([0, 1], [0, 1], 'k--', lw=2)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curves')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Calculate metrics
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb, average='weighted', zero_division=0)
recall_xgb = recall_score(y_test, y_pred_xgb, average='weighted')
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')
kappa_xgb = cohen_kappa_score(y_test, y_pred_xgb)
roc_auc_xgb = roc_auc_score(label_binarize(y_test, classes=[0, 1, 2, 3]), y_score_xgb, average='macro', multi_class='ovr')

# Output the metrics
print(f"Accuracy: {accuracy_xgb:}")
print(f"Precision: {precision_xgb:}")
print(f"Recall: {recall_xgb:}")
print(f"F1 Score: {f1_xgb:}")
print(f"Cohen's Kappa: {kappa_xgb:}")
print(f"ROC-AUC: {roc_auc_xgb:}")

#### 4) Neural Network

In [ ]:
# Define the model structure
model_nn = Sequential([
    Dense(10, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(4, activation='softmax')
])

model_nn.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

y_train_encoded = to_categorical(y_train, num_classes=4)
y_test_encoded = to_categorical(y_test, num_classes=4)

# Retrain the model with the one-hot encoded targets

model_nn.fit(X_train, y_train_encoded, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set with one-hot encoded targets
loss, accuracy = model_nn.evaluate(X_test, y_test_encoded)

In [ ]:
# Get probability distributions for each class
y_prob_nn = model_nn.predict(X_test)

y_pred = np.argmax(y_prob_nn, axis=1)

if y_test.ndim > 1:
    y_test = np.argmax(y_test, axis=1)


# Convert y_test to binary format for ROC AUC
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))

# Calculate metrics
accuracy_nn = accuracy_score(y_test, y_pred)
precision_nn = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall_nn = recall_score(y_test, y_pred, average='weighted')
f1_nn = f1_score(y_test, y_pred, average='weighted')
kappa_nn = cohen_kappa_score(y_test, y_pred)
roc_auc_nn = roc_auc_score(y_test_bin, y_prob_nn, average='macro', multi_class='ovr')

# Output the metrics
print(f"Accuracy: {accuracy_nn:}")
print(f"Precision: {precision_nn:}")
print(f"Recall: {recall_nn:}")
print(f"F1 Score: {f1_nn:}")
print(f"Cohen's Kappa: {kappa_nn:}")
print(f"ROC-AUC: {roc_auc_nn:}")

#### 5) CatBoost

In [ ]:
# Initialize the CatBoost classifier
catboost_model = CatBoostClassifier(iterations=500,
                                     learning_rate=0.1,
                                     depth=6,
                                     random_seed=42,
                                     verbose=100)

# Train the model
catboost_model.fit(X_train, y_train)

# Make predictions
y_pred_Cat = catboost_model.predict(X_test)
y_prob_Cat = catboost_model.predict_proba(X_test)

In [ ]:
# Feature Importance Plot
feature_importances = catboost_model.feature_importances_
feature_names = X.columns

# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Feature Importance from CatBoost')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_Cat)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# SHAP values for model interpretability
explainer = shap.TreeExplainer(catboost_model)
shap_values = explainer.shap_values(X_test.iloc[:100,:])
shap.summary_plot(shap_values, X_test.iloc[:100,:], plot_type="bar")

In [ ]:
# Binarize y_test if multiclass
y_test_bin = label_binarize(y_test, classes=[0, 1, 2, 3])
fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_prob_Cat.ravel())
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
n_classes = 4

# Store FPR, TPR, and AUC for each class
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_prob_Cat[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'purple']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

# Plot the diagonal (random classifier)
plt.plot([0, 1], [0, 1], 'k--', lw=2)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curves')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Calculate metrics for CatBoost model
accuracy_Cat = accuracy_score(y_test, y_pred_Cat)
precision_Cat = precision_score(y_test, y_pred_Cat, average='weighted', zero_division=0)
recall_Cat = recall_score(y_test, y_pred_Cat, average='weighted')
f1_Cat = f1_score(y_test, y_pred_Cat, average='weighted')
kappa_Cat = cohen_kappa_score(y_test, y_pred_Cat)
roc_auc_Cat = roc_auc_score(label_binarize(y_test, classes=[0, 1, 2, 3]), y_prob_Cat, average='macro', multi_class='ovr')

# Output the metrics
print(f"Accuracy: {accuracy_Cat:}")
print(f"Precision: {precision_Cat:}")
print(f"Recall: {recall_Cat:}")
print(f"F1 Score: {f1_Cat:}")
print(f"Cohen's Kappa: {kappa_Cat:}")
print(f"ROC-AUC: {roc_auc_Cat:}")

#### 6) LightGBM

In [ ]:
# Initialize the LightGBM classifier
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=31,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

# Train the model
lgb_model.fit(X_train, y_train)

# Make predictions
y_pred_LGB = lgb_model.predict(X_test)
y_prob_LGB = lgb_model.predict_proba(X_test)

In [ ]:
# Feature Importance Plot
feature_importances = lgb_model.feature_importances_
feature_names = X.columns

# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Feature Importance from LightGBM')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_LGB)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# SHAP values for model interpretability
explainer = shap.TreeExplainer(lgb_model)
shap_values = explainer.shap_values(X_test.iloc[:100,:])
shap.summary_plot(shap_values, X_test.iloc[:100,:], plot_type="bar")

In [ ]:
# Binarize y_test if multiclass
y_test_bin = label_binarize(y_test, classes=[0, 1, 2, 3])
fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_prob_LGB.ravel())
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
n_classes = 4

# Store FPR, TPR, and AUC for each class
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_prob_LGB[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'purple']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

# Plot the diagonal (random classifier)
plt.plot([0, 1], [0, 1], 'k--', lw=2)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curves')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Binarize labels for ROC-AUC
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))

# Metrics calculation
accuracy_LGB = accuracy_score(y_test, y_pred_LGB)
precision_LGB = precision_score(y_test, y_pred_LGB, average='weighted')
recall_LGB = recall_score(y_test, y_pred_LGB, average='weighted')
f1_LGB = f1_score(y_test, y_pred_LGB, average='weighted')
kappa_LGB = cohen_kappa_score(y_test, y_pred_LGB)
roc_auc_LGB = roc_auc_score(y_test_bin, y_prob_LGB, multi_class='ovr')

# Output metrics
print(f"Precision: {precision_LGB:}")
print(f"Recall: {recall_LGB:}")
print(f"F1 Score: {f1_LGB:}")
print(f"Cohen's Kappa: {kappa_LGB:}")
print(f"ROC-AUC: {roc_auc_LGB:}")
print(f"LightGBM Accuracy: {accuracy_LGB:}")

#### 7) Support Vector Machines

In [ ]:
svm_classifier = SVC(kernel='linear', probability=True, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm_classifier.fit(X_train_scaled, y_train)


pca = PCA(n_components=7)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

svm_classifier.fit(X_train_pca, y_train)


In [ ]:
# Create an SVM classifier with one-vs-rest strategy

svm_classifier.fit(X_train_scaled, y_train)

# Predict
y_pred_svm = svm_classifier.predict(X_test_scaled)

y_score_svm = svm_classifier.predict_proba(X_test)


In [ ]:
# Feature Importance (only for linear kernel)
if svm_classifier.kernel == 'linear':
    feature_importances = np.abs(svm_classifier.coef_).sum(axis=0)
    feature_names = X.columns
    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
    plt.title('Feature Importance from SVM')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_svm)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.title('Confusion Matrix for SVM')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# ROC
# Binarize y_test if multiclass
y_test_bin = label_binarize(y_test, classes=[0, 1, 2, 3])
fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_score_svm.ravel())
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
n_classes = 4

# Store FPR, TPR, and AUC for each class
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score_svm[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'purple']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

# Plot the diagonal (random classifier)
plt.plot([0, 1], [0, 1], 'k--', lw=2)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curves')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Metrics

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm, average='weighted', zero_division=0)
recall_svm = recall_score(y_test, y_pred_svm, average='weighted')
f1_svm = f1_score(y_test, y_pred_svm, average='weighted')
kappa_svm = cohen_kappa_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test_bin, y_score_svm, multi_class='ovr')

print(f"Accuracy: {accuracy_svm}")
print(f"Precision: {precision_svm}")
print(f"Recall: {recall_svm}")
print(f"F1 Score: {f1_svm}")
print(f"Cohen's Kappa: {kappa_svm:}")
print(f"ROC-AUC: {roc_auc_svm:}")

#### 8) Ensemble Method

In [93]:
# Define the base learners
base_learners = [
    ('xgb', xgb_model),
    ('cat', catboost_model),
    ('light', lgb_model)
]

# Define the meta-learner
meta_learner = GradientBoostingClassifier(n_estimators=50, learning_rate=0.1, random_state=42)

In [ ]:
# Initialize the Stacking Classifier with a classification meta-learner
stacked_model = StackingClassifier(estimators=base_learners, final_estimator=meta_learner, cv=5)

# Assume X_train, y_train are your training data
stacked_model.fit(X_train, y_train)

In [ ]:
# Fit the model
stacked_model.fit(X_train, y_train)

# Make predictions
y_pred_ens = stacked_model.predict(X_test)
y_score_ens = stacked_model.predict_proba(X_test)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_ens)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Binarize y_test if multiclass
y_test_bin = label_binarize(y_test, classes=[0, 1, 2, 3])
fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_score_ens.ravel())
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
n_classes = 4

# Store FPR, TPR, and AUC for each class
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i],  y_score_ens[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'purple']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

# Plot the diagonal (random classifier)
plt.plot([0, 1], [0, 1], 'k--', lw=2)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curves')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Calculate metrics
accuracy_ens = accuracy_score(y_test, y_pred_ens)
precision_ens = precision_score(y_test, y_pred_ens, average='weighted', zero_division=0)
recall_ens = recall_score(y_test, y_pred_ens, average='weighted')
f1_ens = f1_score(y_test, y_pred_ens, average='weighted')
kappa_ens = cohen_kappa_score(y_test, y_pred_ens)
roc_auc_ens = roc_auc_score(label_binarize(y_test, classes=[0, 1, 2, 3]), y_score_ens, average='macro', multi_class='ovr')

# Output the metrics
print(f"Accuracy: {accuracy_ens:}")
print(f"Precision: {precision_ens:}")
print(f"Recall: {recall_ens:}")
print(f"F1 Score: {f1_ens:}")
print(f"Cohen's Kappa: {kappa_ens:}")
print(f"ROC-AUC: {roc_auc_ens:}")


#### 9) Models Evaluations

In [ ]:
# Creating DataFrame
metrics_df_1 = pd.DataFrame({
    "Model": ["Baseline", "Random Forest", "XGBoost", "Neural Network","CatBoost", "LightGBM", "SVM", "Ensemble"],
    "Accuracy": [accuracy_base, accuracy_rf, accuracy_xgb, accuracy_nn, accuracy_Cat, accuracy_LGB, accuracy_svm, accuracy_ens],
    "Precision": [precision_base, precision_rf, precision_xgb, precision_nn, precision_Cat, precision_LGB, precision_svm, precision_ens],
    "Recall": [recall_base, recall_rf, recall_xgb, recall_nn, recall_Cat, recall_LGB, recall_svm, recall_ens],
    "F1 Score": [f1_base,f1_rf, f1_xgb, f1_nn, f1_Cat, f1_LGB, f1_svm, f1_ens],
})

metrics_df_2 = pd.DataFrame({
    "Model": ["Baseline","Random Forest", "XGBoost", "Neural Network","CatBoost","LightGBM", "SVM","Ensemble"],
    "Cohen's Kappa": [kappa_base,kappa_rf, kappa_xgb, kappa_nn, kappa_Cat, kappa_LGB, kappa_svm, kappa_ens],
    "ROC-AUC": [roc_auc_base, roc_auc_rf, roc_auc_xgb, roc_auc_nn, roc_auc_Cat, roc_auc_LGB, roc_auc_svm, roc_auc_ens]
})
# Display the DataFrame
print(metrics_df_1)
print("")
print(metrics_df_2)

#####  Top 3 models based on their performance

In [ ]:
# Merging both DataFrames
metrics_combined = pd.merge(metrics_df_1, metrics_df_2, on="Model")

# Adding a rank column based on Accuracy, Precision, Recall, F1 Score, Cohen's Kappa, and ROC-AUC
metrics_combined["Average Rank"] = metrics_combined[
    ["Accuracy", "Precision", "Recall", "F1 Score", "Cohen's Kappa", "ROC-AUC"]
].mean(axis=1)

# Sorting by Average Rank to find the best model
metrics_combined = metrics_combined.sort_values(by="Average Rank", ascending=False)

# Display only the top 3 models
top_3_models = metrics_combined.head(3)
print(top_3_models)